# MPI for Sonar simulation

## Imports

In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')

In [2]:
%%px --no-stream --group-outputs=engine

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"Hi, I'm rank {rank}")

[stdout:0] 
Hi, I'm rank 0
[stdout:1] 
Hi, I'm rank 1


In [3]:
%%px --no-stream --group-outputs=engine

import sys
import os
import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, os.path.abspath('/home/hajta2/study/sonar-FWI/cli/'))
from simulation.sonar import Sonar
from simulation.utils import EllipsisBottom, gen_velocity_profile
from simulation.sources import GaborSource

In [5]:
%%px --no-stream --group-outputs=engine
from devito import configuration
configuration['mpi'] = 'full'
configuration['language'] = 'openmp'

## Initialization

In [6]:
%%px --no-stream --group-outputs=engine

domain_size = (60, 30)
v_env = 1.5
ns = 128
source_distance = 0.002
f0 = 5
space_order = 8
spatial_dist = round(v_env / f0 / 3, 6)
dt = spatial_dist / 20

In [7]:
%%px --no-stream --group-outputs=engine

sonar = Sonar(domain_size, f0, v_env, EllipsisBottom(True), source_distance=source_distance, ns=ns, spatial_dist=spatial_dist)
sonar.set_source()
sonar.finalize()

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

[stderr:0] 
51it [00:00, 1482.21it/s]
Operator `initdamp` ran in 0.01 s
[stderr:1] 
51it [00:00, 1664.37it/s]
Operator `initdamp` ran in 0.01 s


In [9]:
%%px

src = sonar.src
alpha = 45
dt = sonar.model.critical_dt
ns = src.coordinates.data.shape[0]

if alpha <= 90:
    max_latency = (
        np.cos(np.deg2rad(alpha)) * ((ns - 1) * source_distance / v_env) / dt
    )
elif alpha > 90:
    max_latency = np.cos(np.deg2rad(alpha)) * (source_distance / v_env) / dt
for i in range(ns):
    latency = -np.cos(np.deg2rad(alpha)) * (i * source_distance / v_env)
    src.data[:, i] = np.roll(np.array(src.data[:, i]), int(latency / dt + max_latency))
sonar.u.data.fill(0)
sonar.op(time=sonar.time_range.num - 2, dt=dt)

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

Received Keyboard Interrupt. Sending signal SIGINT to engines...


In [11]:
%%px
all_rec_data = comm.gather(sonar.rec.data, root=0)

# If current process is root
if rank == 0:
    # Concatenate the gathered data if needed
    full_rec_data = np.concatenate(all_rec_data, axis=1)

    # Write the data to file

[stderr:0] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


[stderr:1] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]